In [1]:
!./install.sh --quiet

🍹 Building a mixed python/rust project
🔗 Found pyo3 bindings with abi3 support
🐍 Found CPython 3.13 at /Users/madhavajay/dev/cleon/workspace1/.venv/bin/python
📡 Using build options features from pyproject.toml
💻 Using `MACOSX_DEPLOYMENT_TARGET=11.0` for aarch64-apple-darwin by default
📦 Including files matching "src/cleon/py.typed"
📦 Including files matching "src/cleon/_cleon.pyi"
📦 Built wheel for abi3 Python ≥ 3.9 to /Users/madhavajay/dev/cleon/workspace1/dist/cleon-0.1.0-cp39-abi3-macosx_11_0_arm64.whl


In [2]:
import cleon

In [3]:
cleon.use("codex", debug=True)

Codex session started.



Add a `>` before your prompt:  
                 
> &gt; Whats your name?  
I am Codex!                   


In [4]:
> Whats your name?

'❌ Codex error: cleon session stopped. Start a new session with cleon.use(..., session_id=...) to resume or cleon.use(...) to start fresh.'

In [ ]:
cleon.stop()

In [ ]:
> Whats your name?

In [ ]:
x = cleon.stop()

In [ ]:
x

In [ ]:
# cleon.auth("codex")

In [ ]:
print("hello banana")

In [ ]:
> can you clone https://github.com/karpathy/micrograd to this folder?

In [ ]:
> okay can you read all the code in ./micrograd

In [ ]:
> give me the first step of a tutorial on implementing micrograd from scratch to learn how it works

In [ ]:
from dataclasses import dataclass

In [ ]:
from typing import Tuple

In [ ]:
from dataclasses import field

In [ ]:
from collections.abc import Callable

In [ ]:
@dataclass
class Value:
    data: float = None
    parent: Tuple = field(default_factory=tuple)
    grad: float = 0.0
    _op: str = ""
    _backward: Callable = field(default_factory=lambda: lambda: None)

    def __repr__(self):
        return f"<{self._op}: {self.data}>"

In [ ]:
> what do you think of my first code?

In [ ]:
a = Value(2.0, parent=(), _op='leaf')
b = Value(3.0, parent=(), _op='leaf')
c = Value(a.data + b.data, parent=(a, b), _op='+')
print(a, b, c)  # <leaf: 2.0> <leaf: 3.0> <+: 5.0>

In [ ]:
@dataclass
class Value:
    data: float = None
    parent: Tuple = field(default_factory=tuple)
    grad: float = 0.0
    _op: str = ""
    _backward: Callable = field(default_factory=lambda: lambda: None)

    def __repr__(self):
        return f"<{self._op}: {self.data}>"

    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(data=other)
        out = Value(
            data=self.data + other.data,
            parent=(self, other),
            grad=0.0,
            _op="+"
        )

        def _backward():
            self.grad += out.grad
            other.grad += out.grad
        out._backward = _backward

        return out

In [ ]:
> what do you think of that __add__ method?

In [ ]:
@dataclass
class Value:
    data: float = None
    parent: Tuple = field(default_factory=tuple)
    grad: float = 0.0
    _op: str = ""
    _backward: Callable = field(default_factory=lambda: lambda: None)

    def __repr__(self):
        return f"<{self._op}: {self.data}>"

    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(data=other)
        out = Value(
            data=self.data + other.data,
            parent=(self, other),
            grad=0.0,
            _op="+"
        )

        def _backward():
            self.grad += out.grad
            other.grad += out.grad
        out._backward = _backward

        return out

    def backward(self):
        self.grad = 1.0
        nodes, visited = [], set()

        def dfs(node):
            if id(node) not in visited:
                visited.add(id(node))
                for p in node.parent:
                    dfs(p)
                nodes.append(node)
        dfs(self)

        for node in reversed(nodes):
            print(f"Running {node}.backward()")
            node._backward()

In [ ]:
a = Value(2.0, parent=(), _op='leaf')
b = Value(3.0, parent=(), _op='leaf')
c = Value(a.data + b.data, parent=(a, b), _op='+')
print(a, b, c)  # <leaf: 2.0> <leaf: 3.0> <+: 5.0>

In [ ]:
c.backward()

In [ ]:
> is that it?

In [ ]:
> how can i test it just with add and know its correct

In [ ]:
a = Value(2)
b = Value(3)
c = a + b
c.backward()

assert a.grad == 1
assert b.grad == 1

In [ ]:
a = Value(2)
b = Value(3)
c = a + b + b
c.backward()

assert a.grad == 1
assert b.grad == 2

In [ ]:
> looks right, can you explain the theory on what we just did

In [ ]:
> Why is grad 0 by default and 1 for when i call backward? would that only be true for addition?

In [ ]:
> why is b 2 in that second test?

In [ ]:
> okay what should i do next to learn how to reimplement micrograd?

In [ ]:
> okay lets add __mul__ what is different between __add__ and __mul__ ?